In [ ]:
import sys

sys.path.append("../")

from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder

import tensorflow as tf
import numpy as np
SEED = 42
np.random.seed(SEED) 
tf.random.set_seed(SEED)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
from utils import load_json, dump_json

from loaders.LHCO import LhcoRnDLoader

loader = LhcoRnDLoader.from_json("../configs/loader/default_rnd.json")
dataset_cfg = load_json("../configs/loader/default_dataset.json")
dataset = loader.make_full_dataset(**dataset_cfg)

In [ ]:
import tensorflow.keras as tfk
import tensorflow as tf

from models.autoencoder import DenseAutoencoder
from models.flows import CondMAF, MAF

builder = PaeBuilder()

pae, ae_train, nf_train = builder.from_json('../configs/pae/trial.json')

In [ ]:
task = HLFAnalysis(pae, dataset=dataset)


In [ ]:
task.reweighting(estimator='gmm', fit_key='mjj_train')

In [ ]:
task.make_cond_inputs(['mjj_train', 'mjj_test', 'mjj_valid'])
task.train(ae_train,nf_train)

In [ ]:
r = task.evaluate(prc = 90)

In [ ]:
q = {
 'id': [],
 'js_div_train': [],
 'js_div_test': [],
 'sig_eff': [],
 'bkg_rej': [],
 'auc': []}

In [ ]:
for key in r.keys():
    q[key].append(r[key])

q

In [ ]:
task.plot_training()

In [ ]:
task.plot_latent_space()